In [16]:
import cv2
import os

# we have created Function to create directory if it doesn't exist
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# it is Function to capture images from webcam, detect faces, crop them, and save
def capture_images(label_name):
    # Check if the label name is 'q'
    if label_name.lower() == 'q':
        print("Exiting the program.")
        return
    
    # Create directory for the label (person's name)
    create_directory('dataset/' + label_name)
    
    # Load pre-trained face detection model
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Start capturing video from webcam
    cap = cv2.VideoCapture(0)
    
    # Check if the webcam is opened successfully
    if not cap.isOpened():
        print("Error: Failed to open webcam.")
        return

    count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame.")
            break
        
        # Convert frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        # Draw rectangles around detected faces
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Display the frame
        cv2.imshow('Capturing Images - Press "c" to capture (Press "q" to quit)', frame)

        # Press 'c' to capture photo
        key = cv2.waitKey(1)
        if key == ord('c'):
            for (x, y, w, h) in faces:
                # Crop the face
                face_roi = frame[y:y+h, x:x+w]

                # Save the cropped face in the corresponding label folder
                img_path = f'dataset/{label_name}/img_{count}.jpg'
                cv2.imwrite(img_path, face_roi)
                count += 1
                print(f"Image {count} captured")

                # Draw rectangle around the captured face
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Press 'q' to quit
        if key == ord('q'):
            break

        # Limit to 200 images
        if count >= 200:
            break

    # Release the camera
    cap.release()
    cv2.destroyAllWindows()

# Main function
def main():
    # Input label name (person's name)
    label_name = input("Enter the name of the person (Enter 'q' to quit): ")

    # Capture images, detect faces, crop them, and save
    capture_images(label_name)

if __name__ == "__main__":
    main()


Enter the name of the person (Enter 'q' to quit): pratik
Image 1 captured
Image 2 captured
Image 3 captured
Image 4 captured
Image 5 captured
Image 6 captured
Image 7 captured
Image 8 captured
Image 9 captured
Image 10 captured
Image 11 captured
Image 12 captured
Image 13 captured
Image 14 captured
Image 15 captured
Image 16 captured
Image 17 captured
Image 18 captured
Image 19 captured
Image 20 captured
Image 21 captured
Image 22 captured
Image 23 captured
Image 24 captured
Image 25 captured
Image 26 captured
Image 27 captured
Image 28 captured
Image 29 captured
Image 30 captured
Image 31 captured
Image 32 captured
Image 33 captured
Image 34 captured
Image 35 captured
Image 36 captured
Image 37 captured
Image 38 captured
Image 39 captured
Image 40 captured
Image 41 captured
Image 42 captured
Image 43 captured
Image 44 captured
Image 45 captured
Image 46 captured
Image 47 captured
Image 48 captured
Image 49 captured
Image 50 captured
Image 51 captured
Image 52 captured
Image 53 capture

In [17]:
import os
import csv

# Function to collect images and create dataset
def create_dataset(dataset_directory):
    # Open or create a CSV file to store image paths and labels
    with open('dataset.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['image_path', 'label'])

        # Traverse the dataset directory to collect images and assign labels
        for label_name in os.listdir(dataset_directory):
            label_directory = os.path.join(dataset_directory, label_name)
            if os.path.isdir(label_directory):
                for image_name in os.listdir(label_directory):
                    image_path = os.path.join(label_directory, image_name)
                    # Write image path and label to the CSV file
                    csv_writer.writerow([image_path, label_name])

def start():
    # Directory to save the dataset
    dataset_directory = 'dataset'

    # Create the directory if it doesn't exist
    if not os.path.exists(dataset_directory):
        os.makedirs(dataset_directory)

    # Collect images and create dataset
    create_dataset(dataset_directory)

    # Your other code here...


start()

In [18]:
import os
import csv
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from joblib import dump

# Function to load dataset
def load_dataset(dataset_file):
    X = []
    y = []

    with open(dataset_file, 'r') as file:
        lines = file.readlines()
        for line in lines[1:]:
            image_path, label = line.strip().split(',')
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            # Resize the image to match the input size of the model
            resized_image = cv2.resize(image, (160, 160))
            X.append(resized_image.flatten())
            y.append(label)

    return np.array(X), np.array(y)


def start():
    # Directory to save the dataset
    dataset_directory = 'dataset'

    # Create the dataset


    # Path to the dataset CSV file
    dataset_file = 'dataset.csv'

    # Load the dataset
    X, y = load_dataset(dataset_file)

    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize SVM classifier
    clf = SVC(kernel='linear')

    # Train the classifier
    clf.fit(X_train, y_train)

    # Predict labels for test set
    y_pred = clf.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print('Accuracy:', accuracy)

    # Save the trained model to disk
    model_file = 'face_recognition_model.joblib'
    dump(clf, model_file)
    print('Model saved to', model_file)


start()

Accuracy: 1.0
Model saved to face_recognition_model.joblib


In [19]:
import cv2
from joblib import load

def start():
    # Load the trained model from disk
    model_file = 'face_recognition_model.joblib'
    clf = load(model_file)

    # Initialize the webcam or video capture device
    video_capture = cv2.VideoCapture(0)

    # Load the pre-trained face cascade classifier
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    while True:
        # Capture frame-by-frame
        ret, frame = video_capture.read()

        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Iterate over each detected face
        for (x, y, w, h) in faces:
            # Extract the face region from the frame
            face_roi = gray[y:y+h, x:x+w]

            # Resize the face region to match the input size of the model
            face_roi_resized = cv2.resize(face_roi, (160, 160))

            # Flatten the face region
            face_features = face_roi_resized.flatten()

            # Predict the label (name) of the person
            predicted_label = clf.predict([face_features])[0]

            # Draw a rectangle around the detected face and display the predicted label
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, predicted_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

        # Display the resulting frame
        cv2.imshow('Video', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture device and close all OpenCV windows
    video_capture.release()
    cv2.destroyAllWindows()


start()